In [3]:

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.model_selection import train_test_split

eurosatNTData = torchvision.datasets.EuroSAT(root = './data', download = True, transform = None)
device = "cuda" if torch.cuda.is_available() else "cpu"
useAugmentation = False


100%|██████████| 94.3M/94.3M [00:00<00:00, 318MB/s]


In [1]:
def GetLoaders(isAugmented = False, batchSize = 32):

  indices = list(range(len(eurosatNTData)))
  labels = eurosatNTData.targets

  trainIdx, testIdx = train_test_split(
      indices,
      test_size=0.1,
      random_state=42,
      stratify=labels
  )

  noAUGTransform = transforms.Compose([
      transforms.ToTensor(),
      transforms.Normalize(mean = [0.5], std = [0.5])
  ])

  AUGTransform = transforms.Compose([
      transforms.RandomHorizontalFlip(),
      transforms.ToTensor(),
      transforms.Normalize(mean = [0.5], std = [0.5])
  ])


  if(isAugmented):
    notAugmentedTrain = Subset(torchvision.datasets.EuroSAT(root="./data", transform=noAUGTransform),trainIdx)
    augmentedTrain = Subset(torchvision.datasets.EuroSAT(root="./data", transform=AUGTransform),trainIdx)
    trainDataset = torch.utils.data.ConcatDataset([augmentedTrain,notAugmentedTrain])
  else:
    trainDataset = Subset(torchvision.datasets.EuroSAT(root="./data", transform=noAUGTransform),trainIdx)

  testDataset = Subset(torchvision.datasets.EuroSAT(root="./data", transform=noAUGTransform),testIdx)
  trainLoader = torch.utils.data.DataLoader(trainDataset, batch_size = batchSize, shuffle = True, num_workers = 4, pin_memory=True)
  testLoader = torch.utils.data.DataLoader(testDataset, batch_size = batchSize, shuffle = False)

  return trainLoader, testLoader


In [4]:
trainLoader, testLoader = GetLoaders(useAugmentation, 32)

In [5]:
class CNN(nn.Module):
  def __init__(self, inChannels, numClasses):
    super(CNN, self).__init__()
    self.conv1 = nn.Conv2d(in_channels = inChannels, out_channels= 16, kernel_size = 3, stride = 1, padding = 1)
    self.pool = nn.MaxPool2d(kernel_size = 2, stride = 2)
    self.conv2 = nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size = 3, stride = 1, padding = 1)
    self.fc1 = nn.Linear(32 * 16 * 16, 128)
    self.fc2 = nn.Linear(128, numClasses)
    self.dropfc1 = nn.Dropout(p=0.2)
    # self.dropconv2 = nn.Dropout2d(p=0.1)

  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = self.pool(x)
    x = F.relu(self.conv2(x))
    x = self.pool(x)
    x = x.view(-1, 32 * 16 * 16)
    x = F.relu(self.dropfc1(self.fc1(x)))
    x = self.fc2(x)
    return x

model = CNN(inChannels = 3, numClasses = 10).to(device)

In [6]:
#Hyper parameters
learningRate = 1e-3
epochs = 70
lossFunction = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learningRate)
scheduler = lr_scheduler.LinearLR(optimizer,start_factor = 1.0, end_factor= 0.1, total_iters= 50)
losses = []

In [7]:
def train():
  for epoch in range(epochs):
    epochLoss = 0
    numBatches = 0
    for X, y in trainLoader:
      X = X.to(device, non_blocking=True)
      y = y.to(device, non_blocking=True)
      optimizer.zero_grad()
      pred = model(X)
      loss = lossFunction(pred, y)
      loss.backward()
      optimizer.step()
      epochLoss += loss.item()
      numBatches += 1
    scheduler.step()
    avgLoss = epochLoss/numBatches
    losses.append(avgLoss)
    print(f'Epoch {epoch+1}, Loss: {avgLoss:.9f}, Lr: {optimizer.param_groups[0]["lr"]:.6}')

train()

Epoch 1, Loss: 1.031725361, Lr: 0.000982
Epoch 2, Loss: 0.697134058, Lr: 0.000964
Epoch 3, Loss: 0.571193056, Lr: 0.000946
Epoch 4, Loss: 0.491166171, Lr: 0.000928
Epoch 5, Loss: 0.428205322, Lr: 0.00091
Epoch 6, Loss: 0.356834076, Lr: 0.000892
Epoch 7, Loss: 0.310447097, Lr: 0.000874
Epoch 8, Loss: 0.259895958, Lr: 0.000856
Epoch 9, Loss: 0.221902967, Lr: 0.000838
Epoch 10, Loss: 0.201382188, Lr: 0.00082
Epoch 11, Loss: 0.173202672, Lr: 0.000802
Epoch 12, Loss: 0.151418368, Lr: 0.000784
Epoch 13, Loss: 0.130801942, Lr: 0.000766
Epoch 14, Loss: 0.113939860, Lr: 0.000748
Epoch 15, Loss: 0.107648733, Lr: 0.00073
Epoch 16, Loss: 0.094370685, Lr: 0.000712
Epoch 17, Loss: 0.086830538, Lr: 0.000694
Epoch 18, Loss: 0.072177010, Lr: 0.000676
Epoch 19, Loss: 0.069534680, Lr: 0.000658
Epoch 20, Loss: 0.071831410, Lr: 0.00064
Epoch 21, Loss: 0.060292879, Lr: 0.000622
Epoch 22, Loss: 0.059476735, Lr: 0.000604
Epoch 23, Loss: 0.050708793, Lr: 0.000586
Epoch 24, Loss: 0.046690852, Lr: 0.000568
Epoch

In [10]:
def check_accuracy():


    num_correct = 0
    num_samples = 0
    model.eval()  # Set the model to evaluation mode

    with torch.no_grad():  # Disable gradient calculation
        for x, y in testLoader:
            x = x.to(device,non_blocking=True)
            y = y.to(device,non_blocking=True)
            # Forward pass: compute the model output
            scores = model(x)
            _, predictions = scores.max(1)  # Get the index of the max log-probability
            num_correct += (predictions == y).sum()  # Count correct predictions
            num_samples += predictions.size(0)  # Count total samples

        # Calculate accuracy
        accuracy = float(num_correct) / float(num_samples) * 100
        print(f"Got {num_correct}/{num_samples} with accuracy {accuracy:.2f}%")

    model.train()  # Set the model back to training mode

# Final accuracy check on training and test sets
check_accuracy()


Got 2364/2700 with accuracy 87.56%
